# TODO
- [X] specify lesson plan in json format, then convert to markdown
- [ ] parallelize the lesson plan creation
- [ ] generate all activity ideas at once, so there is less repetition in the ideas (e.g., there are many small variations on the 'scavenger hunt' idea)
- [ ] try different models:
  - [X] chat model instead of LLM
  - [ ] different providers instead of OpenAI
  - [ ] local model

In [1]:
import os
import json
import time
import pprint as pp
import pandas as pd
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from IPython.display import display, Markdown, JSON

In [2]:
load_dotenv(override=True) # loading API key from environment

True

## Read in data with annual curriculum plan

In [3]:
GSHEET_ID = os.getenv('GSHEET_ID')
google_sheet_url = f"https://docs.google.com/spreadsheets/d/{GSHEET_ID}/export?format=csv&gid=0"
annual_plan_df = pd.read_csv(google_sheet_url)
annual_plan_df.rename(columns=lambda x: x.replace(" ", "_").lower(), inplace=True)
annual_plan_df

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,"Personal narratives, reading comprehension",Counting and sorting,Our classroom community; citizenship and gover...,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",Descriptive writing (My Place); vocabulary dev...,latitutde and longitude; place value,My hometown; local history; geography,"Water cycle, rivers; Geology, caves, aquifers"
2,October,Fall; Halloween; Dia de los Muertos,Creative writing; grammar and punctuation,"units of measurement (length, weight, etc.); E...",Fall celebrations around the world; elections,Food chains and food webs; Local flora and fauna
3,November,Gratitude; Native Americans,Listening and speaking skills; Thankful Though...,Addition and subtraction,Native American history; Settlement of US and ...,Seasonal Changes and Plants; Winter Animal Ada...
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Geometry (2-D shapes); symmetry (snowflake),Winter holidays worldwide,"Physical science (matter, energy, force, motion)"
5,January,"Calendar, New Year, Goals, MLK",Dreams and goals; journaling; 'I Have a Dream'...,"Time units, Calendar math, multiplication",New Year's Celebrations worldwide; Community h...,"Climate zones, seasons, and weather"
6,February,"Black History, Love/Valentine's Day",African American biographies; exploring emotio...,"Fractions, Division; Number Patterns",African American Achievements; Civil Rights Mo...,"The human body, heart health and exercise"
7,March,Spring; Easter; Ramadan,Springtime poetry; nature observation,Patterns and Geometry; Geometry in Islamic Art,"Cultural Spring celebrations -- Easter, Ramadan",Life cycles of plants and animals (frogs and b...
8,April,Space; Passover,"Space stories, science fiction; constellation ...",Patterns and algebraic thinking; math in the s...,Passover origins; History of astronomy,Solar System and celestial bodies
9,May,Ocean Life,Ocean vocabulary; sea creatures; sea stories,Problem-solving and mathematical reasoning; oc...,Coastal ecosystems; exploration and navigation,Ocean zones; Marine life and habitats


# Create Prompt Templates

In [4]:
class ActivityIdea(BaseModel):
    name: str
    description: str

class ActivityIdeasOutput(BaseModel):
    ideas: List[ActivityIdea]
# Set up a parser + inject instructions into the prompt template.
activity_ideas_parser = PydanticOutputParser(pydantic_object=ActivityIdeasOutput)

In [5]:
general_system_prompt = SystemMessagePromptTemplate.from_template(
    template= """
    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Try to work in phonics work and literacy development into the language art activities.
    - Try to work in arithmetic and numeracy development into the math activities.
    - Keep in mind the grade level and number of the students when generating your response.
    """
)

In [6]:
idea_system_prompt = SystemMessagePromptTemplate.from_template(
    template= """
    I want you to help me brainstorm activity ideas for my classroom.
    Each activity could also be thought of as a lesson where the kids are learning valuable skills.
    I only need you to give me the activity names and a brief description of each idea.
    {format_instructions}
    
    Here are some additional guidelines for generating these ideas:
    - Be creative in your activity ideas. Avoid common activities that have no direct relation to the lesson content, like scavenger hunts and word searches.
    - Each activity idea should be able to be accomplished in 30 mins-3 hours.
    - Each activity should relate directly to skills and concepts from the relevant academic subject.
    - The description should only be sentence or two and should not simply repeat the theme or the topic. It should describe the nature of the activity.
    """
)

In [7]:
idea_human_prompt = HumanMessagePromptTemplate.from_template(
    template="""
    I want you to generate {num_ideas} activity ideas for the month of {month}. The theme for this month is {theme}.
    The academic subject is {subject} and the topic suggestions for this month are: {topic}.    
    """
)

In [8]:
idea_chat_prompt_template = ChatPromptTemplate(
    messages=[general_system_prompt, idea_system_prompt, idea_human_prompt],
    input_variables=["num_ideas", "month", "theme", "subject", "topic"],
    partial_variables={"format_instructions": activity_ideas_parser.get_format_instructions()}
)

In [9]:
print(activity_ideas_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ideas": {"title": "Ideas", "type": "array", "items": {"$ref": "#/definitions/ActivityIdea"}}}, "required": ["ideas"], "definitions": {"ActivityIdea": {"title": "ActivityIdea", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}}, "required": ["name", "description"]}}}
```


In [10]:
class LessonPlanOutput(BaseModel):
    time_estimate: str
    list_of_materials: List[str]
    procedure: List[str]
    additional_notes: List[str]
    differentiation_strategies: List[str]

# Set up a parser + inject instructions into the prompt template.
lesson_plan_parser = PydanticOutputParser(pydantic_object=LessonPlanOutput)

In [11]:
lesson_plan_system_prompt = SystemMessagePromptTemplate.from_template(
    template="""
    I want you to generate a detailed lesson plan. I will give you a name and a short description of the lesson, as well as the academic subject the lesson plan should relate to.
    You will provide:
     - a time estimate
     - a list of materials
     - a step-by-step procedure for executing the lesson.
     - additional notes
     - differentiation strategies for evaluating students and providing additional challenge or assistance, and modifications for neurodivergent students
     
     Additional guidelines for the lesson plan:
      - Ensure that the lesson uses and develops skills relevant to the academic subject.
      - Try to make the best estimate of the total amount of time the activity will take. Be generous because these are young kids and they take a while!
      - The 'additional notes' specifies the actual content mentioned in the 'procedure' (e.g., interview questions, puzzle clues, specific writing prompts, etc.), so lesson plan provides everything needed to execute the lesson.
    
    {format_instructions}
    """
)

In [12]:
lesson_plan_human_prompt = HumanMessagePromptTemplate.from_template(
    template="""
    I want you to generate a *detailed* lesson plan for the academic subject of {subject}.
    The name of the lesson is {activity_name}, and the short description is: {activity_description}.
    """
)

In [13]:
    # - The 'differentiation strategies' provides modifications for students who need more support, for students who need more challenge, and for neurodivergent students.

In [14]:
    # - The 'additional notes' must **NOT** include differentiation strageties. There should be no overlap. Double-check this before giving me your response.

In [15]:
lesson_plan_chat_prompt_template = ChatPromptTemplate(
    messages=[general_system_prompt, lesson_plan_system_prompt, lesson_plan_human_prompt],
    input_variables=["activity_name", "activity_description", "subject"],
    partial_variables={"format_instructions": lesson_plan_parser.get_format_instructions()}
)

In [16]:
print(lesson_plan_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"time_estimate": {"title": "Time Estimate", "type": "string"}, "list_of_materials": {"title": "List Of Materials", "type": "array", "items": {"type": "string"}}, "procedure": {"title": "Procedure", "type": "array", "items": {"type": "string"}}, "additional_notes": {"title": "Additional Notes", "type": "array", "items": {"type": "string"}}, "differentiation_strategies": {"title": "Differentiation Strategies", "type": "array", "items": {"type": "string"}}}, "required": ["time_estimate", "list_of_materials", "procedure", "addition

# Test on Single Record

In [17]:
num_plans = 5

In [18]:
month = "September"
subject = 'math'

In [19]:
month_record = annual_plan_df.query("month == @month").to_dict(orient='records')[0]

In [20]:
theme = month_record['theme_to_explore']
topic = month_record[subject]

In [21]:
idea_prompt = idea_chat_prompt_template.format_prompt(
    num_ideas=num_plans,
    month=month,
    theme=theme,
    subject=subject, 
    topic=topic 
)

In [22]:
idea_messages = idea_prompt.to_messages()

In [23]:
idea_messages

[SystemMessage(content='\n    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.\n    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.\n    - Try to work in phonics work and literacy development into the language art activities.\n    - Try to work in arithmetic and numeracy development into the math activities.\n    - Keep in mind the grade level and number of the students when generating your response.\n    ', additional_kwargs={}),
 SystemMessage(content='\n    I want you to help me brainstorm activity ideas for my classroom.\n    Each activity could also be thought of as a lesson where the kids are learning valuable skills.\n    I only need you to give me the activity names and a brief description of each idea.\n    The output should be formatted as a JSON instance that conforms to the JSON schema below

### Instantiate Test Chat Model

In [24]:
model_name = "gpt-3.5-turbo"
temperature = 0.1
chat_model = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)
# model_name = "text-davinci-003"
# model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)
# from langchain.chat_models import ChatGooglePalm
# chat_model = ChatGooglePalm(model_name='models/chat-bison-001', temperature=.8, max_tokens=500)

In [25]:
idea_response = chat_model(idea_messages)

In [26]:
ideas_parsed = activity_ideas_parser.parse(idea_response.content)

In [27]:
pp.pprint(ideas_parsed.dict())

{'ideas': [{'description': 'Students will create a visual representation of '
                           'their personal journey by plotting their '
                           'birthplace and current location on a world map, '
                           'using latitude and longitude coordinates.',
            'name': 'Mapping Our Journey'},
           {'description': 'Students will work in pairs to search for hidden '
                           'objects in the schoolyard, using latitude and '
                           'longitude coordinates as clues to find each item.',
            'name': 'Latitude and Longitude Scavenger Hunt'},
           {'description': 'Students will solve place value puzzles by '
                           'arranging number cards to create the largest and '
                           'smallest numbers possible, reinforcing their '
                           'understanding of place value concepts.',
            'name': 'Place Value Puzzles'},
           {'descrip

In [28]:
idea = ideas_parsed.ideas[0]

In [29]:
idea_name = idea.name
idea_description = idea.description

In [30]:
print(f"{idea_name}: {idea_description}")

Mapping Our Journey: Students will create a visual representation of their personal journey by plotting their birthplace and current location on a world map, using latitude and longitude coordinates.


In [31]:
lesson_plan_prompt_value = lesson_plan_chat_prompt_template.format_prompt(
    activity_name=idea_name,
    activity_description=idea_description,
    subject=subject
)

In [32]:
lp_messages = lesson_plan_prompt_value.to_messages()

In [33]:
lp_messages

[SystemMessage(content='\n    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.\n    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.\n    - Try to work in phonics work and literacy development into the language art activities.\n    - Try to work in arithmetic and numeracy development into the math activities.\n    - Keep in mind the grade level and number of the students when generating your response.\n    ', additional_kwargs={}),
 SystemMessage(content='\n    I want you to generate a detailed lesson plan. I will give you a name and a short description of the lesson, as well as the academic subject the lesson plan should relate to.\n    You will provide:\n     - a time estimate\n     - a list of materials\n     - a step-by-step procedure for executing the lesson.\n     - additional notes\n     - differe

In [34]:
lp_response = chat_model(lp_messages)

In [35]:
lp_parsed = lesson_plan_parser.parse(lp_response.content)

In [36]:
lp_dict = lp_parsed.dict()

In [37]:
full_lp = {k: v for k, v in idea.dict().items()}
full_lp.update(lp_dict)

In [38]:
full_lp['additional_notes']

["Before the lesson, gather the latitude and longitude coordinates for each student's birthplace and current location.",
 'If some students have difficulty with plotting the coordinates, provide additional support by guiding them through the process step by step.',
 'For neurodivergent students, consider providing visual aids or manipulatives to help them understand and visualize the concept of latitude and longitude.',
 'To incorporate literacy development, have the students write a short paragraph describing their personal journey and what they have learned from the activity.']

In [39]:
full_lp['differentiation_strategies']

['For advanced students, challenge them to plot additional locations on the map, such as places they have visited or places they would like to visit in the future.',
 'For students who need extra support, provide a simplified version of the world map with pre-marked latitude and longitude lines to assist with plotting the coordinates.',
 'For neurodivergent students, offer alternative ways of representing their personal journey, such as using pictures or symbols instead of coordinates.']

## Define Markdown Helper Functions

In [40]:
def make_ordered_list_markdown(lst):
    markdown_list = [f"{idx}. {val}" for idx, val in enumerate(lst, start=1)]
    return '\n'.join(markdown_list)

In [41]:
def make_list_markdown(lst):
    return "- " + "\n- ".join(lst)

In [42]:
def convert_lesson_plan_to_markdown(lesson_plan_dict):
    res = []
    res.append(f"## {lesson_plan_dict['name']}")
    res.append(lesson_plan_dict['description'])
    res.append(f"\n### Time estimate: {lesson_plan_dict['time_estimate']}")
    res.append(f"\n### Materials")
    res.append(make_list_markdown(lesson_plan_dict["list_of_materials"]))
    res.append(f"\n### Procedure")
    res.append(make_ordered_list_markdown(lesson_plan_dict['procedure']))
    res.append(f"\n### Additional notes")
    res.append(make_list_markdown(lesson_plan_dict['additional_notes']))
    res.append(f"\n### Differentiation strategies")
    res.append(make_list_markdown(lesson_plan_dict['differentiation_strategies']))
    
    return "\n".join(res)
    

In [43]:
md = convert_lesson_plan_to_markdown(full_lp)

In [44]:
display(Markdown(md))

## Mapping Our Journey
Students will create a visual representation of their personal journey by plotting their birthplace and current location on a world map, using latitude and longitude coordinates.

### Time estimate: 60 minutes

### Materials
- World map
- Latitude and longitude coordinates of birthplace and current location
- Pencils
- Colored markers or crayons

### Procedure
1. Introduce the lesson by explaining to the students that they will be creating a visual representation of their personal journey.
2. Show the students a world map and explain how latitude and longitude coordinates can be used to locate places on the map.
3. Provide each student with a copy of the world map and their latitude and longitude coordinates for their birthplace and current location.
4. Instruct the students to plot their birthplace and current location on the map using the latitude and longitude coordinates.
5. Encourage the students to use different colored markers or crayons to distinguish between their birthplace and current location.
6. Once the students have plotted their locations, have them label each point with the corresponding latitude and longitude coordinates.
7. Give the students time to share their maps with the class and explain the significance of their personal journey.
8. Facilitate a class discussion about the different journeys represented on the maps and the similarities and differences between them.

### Additional notes
- Before the lesson, gather the latitude and longitude coordinates for each student's birthplace and current location.
- If some students have difficulty with plotting the coordinates, provide additional support by guiding them through the process step by step.
- For neurodivergent students, consider providing visual aids or manipulatives to help them understand and visualize the concept of latitude and longitude.
- To incorporate literacy development, have the students write a short paragraph describing their personal journey and what they have learned from the activity.

### Differentiation strategies
- For advanced students, challenge them to plot additional locations on the map, such as places they have visited or places they would like to visit in the future.
- For students who need extra support, provide a simplified version of the world map with pre-marked latitude and longitude lines to assist with plotting the coordinates.
- For neurodivergent students, offer alternative ways of representing their personal journey, such as using pictures or symbols instead of coordinates.

# Define Wrapper Functions

In [45]:
def prompt_wrapper(prompt_template, parser, model, max_retries=5, **kwargs):
    prompt_value = prompt_template.format_prompt(**kwargs)
    messages = prompt_value.to_messages()
    for attempt in range(max_retries + 1):  #sometimes the parser fails inexplicably
        try:
            output = model(messages)
            parsed_output = parser.parse(output.content)
            break
        except Exception as e:
            if attempt < max_retries:
                print(e)
                print(f"Attempt {attempt + 1} failed. Retrying...")
                time.sleep(2)  # Wait before retrying
            else:
                print(f"API call failed after {max_retries} attempts. Skipping.")
    return parsed_output

In [46]:
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=temperature, max_tokens=3000)

In [47]:
# Generate activity ideas for each month-subject combo
activity_ideas_parsed = prompt_wrapper(  
                prompt_template=idea_chat_prompt_template, 
                parser=activity_ideas_parser,
                model=chat_model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )

# examine output
pp.pprint(activity_ideas_parsed.dict())

{'ideas': [{'description': 'Students will create a visual representation of '
                           'their journey to school using latitude and '
                           'longitude coordinates.',
            'name': 'Mapping Our Journey'},
           {'description': 'Students will use a map to identify and locate '
                           'different landmarks in their neighborhood, '
                           'practicing their understanding of latitude and '
                           'longitude.',
            'name': 'Exploring Our Neighborhood'},
           {'description': 'Students will participate in a scavenger hunt '
                           'where they have to find objects or places using '
                           'latitude and longitude coordinates.',
            'name': 'Latitude and Longitude Scavenger Hunt'},
           {'description': 'Students will go on a treasure hunt around the '
                           'school, solving place value problems to find c

In [48]:
def wrapper(annual_plan_df, activity_ideas_model, lesson_plan_model, num_plans=5, months=None):
    res = {}
    if months is None:
        months = annual_plan_df['month'].values
    df = annual_plan_df.query('month.isin(@months)')
    display(df)
    for idx, row in df.head(3).iterrows(): # first iterate through the months in the table
        month = row["month"]
        theme = row["theme_to_explore"]
        print(f"# {month}")
        markdown_lst = []
        markdown_lst.append(f"# {month}")
        markdown_lst.append(f"## Theme: {theme}")
        res[month] = dict()
        subjects = df.columns[2:].values
        for subject in subjects:  # for each month, iterate through the academic subjects
            pretty_subject = subject.replace("_", " ").title()
            markdown_lst.append(f"\n# {pretty_subject}")
            print(f"## {pretty_subject}")
            topic = row[subject]
            activity_ideas_parsed = prompt_wrapper(  # Generate activity ideas for each month-subject combo
                prompt_template=idea_chat_prompt_template, 
                parser=activity_ideas_parser,
                model=activity_ideas_model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )
            print("  - Generated ideas")
            res[month][subject] = activity_ideas_parsed.dict()
            
            for idea in res[month][subject]["ideas"]: #  Generate lesson plans for each idea
                activity_name = idea["name"]
                activity_description = idea["description"]
                lesson_plan = prompt_wrapper(
                    prompt_template=lesson_plan_chat_prompt_template, 
                    parser=lesson_plan_parser,
                    model=lesson_plan_model,
                    activity_name=activity_name, 
                    activity_description=activity_description, 
                    subject=subject
                )
                idea.update(lesson_plan)
                lesson_plan_markdown = convert_lesson_plan_to_markdown(idea)
                markdown_lst.append(f"\n{lesson_plan_markdown}")
                markdown_string = "\n".join(markdown_lst)
            print("  - Generated lesson plans")
                
        # Write the output of each month to a Markdown file
        output_file_path = f'../output/{idx + 1} {month} Lesson Plans.md'
        with open(output_file_path, "w") as output_file:
            output_file.write(markdown_string)

        print(f"Markdown output written to '{output_file_path}'.")
        print("*" * 50, "\n")
        # break
    
    # Write full results to a JSON file
    full_result_output_path = "../output/lesson_plan.json"
    with open(full_result_output_path , "w") as json_file:
        json.dump(res, json_file)
    print(f"JSON output written to '{full_result_output_path }'.")
    return res

# Run

In [49]:
annual_plan_df.head(6)

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,"Personal narratives, reading comprehension",Counting and sorting,Our classroom community; citizenship and gover...,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",Descriptive writing (My Place); vocabulary dev...,latitutde and longitude; place value,My hometown; local history; geography,"Water cycle, rivers; Geology, caves, aquifers"
2,October,Fall; Halloween; Dia de los Muertos,Creative writing; grammar and punctuation,"units of measurement (length, weight, etc.); E...",Fall celebrations around the world; elections,Food chains and food webs; Local flora and fauna
3,November,Gratitude; Native Americans,Listening and speaking skills; Thankful Though...,Addition and subtraction,Native American history; Settlement of US and ...,Seasonal Changes and Plants; Winter Animal Ada...
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Geometry (2-D shapes); symmetry (snowflake),Winter holidays worldwide,"Physical science (matter, energy, force, motion)"
5,January,"Calendar, New Year, Goals, MLK",Dreams and goals; journaling; 'I Have a Dream'...,"Time units, Calendar math, multiplication",New Year's Celebrations worldwide; Community h...,"Climate zones, seasons, and weather"


In [50]:
activity_ideas_model = ChatOpenAI(model_name=model_name, temperature=.8, max_tokens=500)

In [51]:
lesson_plan_model = ChatOpenAI(model_name=model_name, temperature=.2, max_tokens=3000)

In [52]:
months = [
    # "September",
    # "October",
    "November",
    # "December"
]

In [53]:
lesson_plans = wrapper(annual_plan_df, activity_ideas_model, lesson_plan_model, months=months)

,month,theme_to_explore,language_arts,math,social_studies,science
3,November,Gratitude; Native Americans,Listening and speaking skills; Thankful Though...,Addition and subtraction,Native American history; Settlement of US and ...,Seasonal Changes and Plants; Winter Animal Ada...


# November
## Language Arts
  - Generated ideas


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


  - Generated lesson plans
## Math
  - Generated ideas
  - Generated lesson plans
## Social Studies
  - Generated ideas
  - Generated lesson plans
## Science
  - Generated ideas
  - Generated lesson plans
Markdown output written to '../output/4 November Lesson Plans.md'.
************************************************** 

JSON output written to '../output/lesson_plan.json'.


# View Results

In [54]:
month = months[0]

In [55]:
for idea in lesson_plans[month]['language_arts']['ideas']:
    display(Markdown(f"**{idea['name']}**: {idea['description']}"))

**Gratitude Circle**: In this activity, students will sit in a circle and take turns sharing something they are grateful for. This will help develop their listening and speaking skills as they actively listen to their peers and express their own gratitude.

**Gratitude Journal**: Students will create their own gratitude journals, where they can write or draw things they are thankful for. This activity will encourage reflection and improve their language arts skills as they express their thoughts and feelings.

**Thankful Storytelling**: In this activity, students will take turns sharing a story about a time they felt grateful or received an act of kindness. This will enhance their listening and speaking skills while incorporating the theme of gratitude.

**Native American Legends**: Students will explore Native American legends and myths, learning about their storytelling traditions. They will then have the opportunity to create their own short stories inspired by Native American culture. This activity will foster creativity and enhance their language arts skills.

**Gratitude Collaborative Art**: Students will work together to create a collaborative art piece that represents gratitude. They will discuss and share ideas on what gratitude means to them and how they can visually depict it. This activity will promote teamwork and creativity while integrating the theme of gratitude.

In [56]:
display(JSON(lesson_plans))

<IPython.core.display.JSON object>